---
format:
    beamer:
        code-fold: true
        bibliography: introduction.bib
        # titlegraphic: ../data/
        theme: boxes

author: Jacopo Tissino
date: 2024-10-07
---


# Forecasting for the Lunar Gravitational Wave Antenna

> What will be the sensitivity of the LGWA to gravitational waves?

> How many sources will it detect?

> How well will it constrain their parameters?

---

## The Lunar Gravitational Wave Antenna noise curve

![](figures/Detector_strains.png)

---

## Matched filtering

The gravitational-wave **detection** problem: we have data 

$$ d(t) = \underbrace{h_\theta(t)}_{\text{GW}} + \underbrace{
    n _{\text{Gaussian}} (t) + 
    n _{\text{non-Gaussian}} (t)  
}_{\text{noise}\  n(t)}
$$

and we want to find where $h(t)$ is, while typically $|h| \ll |n|$.
The Gaussian component has a spectral density $S_n(f)$.
Let's assume we want to use a **linear filter**, so that in the time domain it reads:

$$ \hat{\rho}(\tau ) = \int d(t + \tau) f(t) \text{d}t
$$

for some function $f(t)$.

---

We want to maximize the "**distinguishability**" of the signal: we can quantify it with the signal-to-noise ratio

$$ \frac{S}{N} = \frac{\hat{\rho} (\text{a signal is present})}{ \text{root-mean-square of } \hat{\rho}}
$$

Ignoring the non-Gaussian part of the noise, the **optimal** solution is $\hat{\rho} \propto (d|h)$, where

$$ (a | b) = 4 \Re \int_0^\infty \frac{a(f) b^*(f)}{S_n(f)}\text{d}f \,,
$$

---

## Optimal signal-to-noise ratio

The **signal-to-noise** ratio statistic is 

$$ \rho = \frac{S}{N} = \frac{(d|h)}{\sqrt{(h|h)}}
$$


With the expected noise realization ($\langle n(t) \rangle = 0$):

$$
\rho _{\text{opt}} = \sqrt{(h|h)} = 2 \sqrt{\int_0^\infty \frac{|h(f)|^2}{S_n(f)} \text{d}f}
\,.
$$ 

If we do not have the data, this is a good proxy. 
For a real detector, we do injection studies and compute a False Alarm Rate (FAR).

---

## SNR thresholds

> What is a "high enough" value for the SNR?

Without time shifts nor non-Gaussianities, the SNR would simply follow
a $\chi^2$ distribution with two degrees of freedom:
"five $\sigma$" significance with a threshold of $\rho = 5.5$. 

In real data this has to be estimated through **injections**:

$$ \text{FAR} = \text{FAR}_8 \exp \left( - \frac{\rho - 8}{\alpha}\right) \,.
$$

For BNS in O1: $\alpha = 0.13$ and $\text{FAR}_8 = 30000 \text{yr}^{-1}$.

---

## Gravitational wave data analysis

Suppose we measure $d = h_\theta + n$, where our model for $h_\theta = h(t; \theta)$ depends on several parameters (typically, between 10 and 15).

We can estimate the parameters $\theta$ by exploring the **posterior distribution** 

$$ p(\theta | d) = \mathcal{L}(d | \theta ) \pi (\theta ) = \mathcal{N} \exp \left( (d | h_\theta) - \frac{1}{2} (h_\theta | h_\theta) \right) \pi (\theta )\,,
$$

where $\pi (\theta )$ is our **prior distribution** on the parameters.
We are neglecting non-Gaussianities in the noise, and assuming its spectral density is known!

---

The posterior is explored **stochastically** (with MCMC, nested sampling...) yielding many samples $\theta_i$ distributed according to $p(\theta | d)$, with which can compute summary statistics:

- mean $\langle \theta_i \rangle$, 
- variance $\sigma_i^2 = \langle (\theta_i- \langle \theta_i\rangle)^2 \rangle$, 
- covariance $\mathcal{C}_{ij} =\langle (\theta_i- \langle \theta_i\rangle) (\theta_j- \langle \theta_j\rangle) \rangle$.

At this stage, we are not making any approximation, and the 
covariance matrix is just a **summary** tool - the full posterior is still available.

---


In [ ]:
from multivariate_normal import MultivariateNormal
n = MultivariateNormal([0, 0], [[1, 0.8], [0.8, 1]])
n.plot_2d_analytical(0, 0, .9)

---

## Antenna pattern

The strain at the detector depends on the antenna pattern:
$$ h(t) = h_{ij} (t) D_{ij}(t) = h_+ (t) F_+ (t) + h_\times (t) F_\times (t) \,.
$$

---

## Fisher matrix

In the Fisher matrix approximation, we are approximating the likelihood as 

$$ \mathcal{L}(d | \theta) \approx \mathcal{N} \exp \left(- \frac{1}{2} \Delta \theta^i \mathcal{F}_{ij} \Delta \theta^j \right)
$$

where $\Delta \theta^i = \theta ^i - \langle \theta ^i \rangle$.

A __multivariate normal distribution__, with covariance matrix $\mathcal{C}_{ij} = \mathcal{F}_{ij}^{-1}$.
This is a good approximation for the posterior in the high-SNR limit, since the prior matters less then.

---

The Fisher matrix $\mathcal{F}_{ij}$ can be computed as the scalar product
of the derivatives of waveforms: 
$$ \mathcal{F}_{ij} = \left.\left\langle \partial_i \partial_j \mathcal{L} \right\rangle \right|_{\theta = \langle \theta \rangle} =  ( \partial_i h | \partial_j h ) = 4 \Re \int_0^{\infty} \frac{1}{S_n(f)}  \frac{\partial h}{\partial \theta _i} \frac{\partial h^*}{\partial \theta _j}\text{d}f\,.
$$


---

For $N$ detectors, 

$$ \mathcal{F}_{ij} = \sum_{k = 1}^N \mathcal{F}_{ij}^{(k)} 
$$

The covariance matrix can be evaluated in seconds, while 
full parameter estimation takes hours to weeks.

Also, it is easy in the Fisher approach to account for new effects such as 
**the rotation of the Earth**.

Tricky step computationally: **inverting** $\mathcal{F}_{ij}$ to get $\mathcal{C}_{ij}$.
